<a href="https://colab.research.google.com/github/javier-fraga-garcia/pagespeed-auditor/blob/main/notebooks/PageSpeed_Auditor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Auditor de Page Speed API

Este cuaderno pretende ser una interfaz de usuario para la realización de auditorías de velocidad de forma automatizada. El objetivo es poder llevar a cabo una gran cantidad de auditorías y que la interacción con el programa sea sencilla. Para ello se ha creado una conexión con la **API de PageSpeed Insights** que permite la realización de auditorías de los **Core Web Vitals** de forma gratuita. Solamente se necesita una clave de API válida y un archivo con las URLs que se desean auditar. En la sección de configuración del entorno se explican los parámetros que es necesario configurar y se proporciona una interfaz para ello. A partir de ahí solo es necesario seguir las instrucciones del cuaderno.

Para más información, o acceso al script de consola de comandos, se puede acceder a este repositorio de [Github](https://github.com/javier-fraga-garcia/pagespeed-auditor).

### Configuración del entorno

In [ ]:
#@markdown #### Configuración de los parámetros para llevar a cabo la auditoría
#@markdown Se deben configurar los siguientes parámetros:
#@markdown * `urls_file` &rarr; la ruta al archivo del que se obtendrán las URLs a auditar. Se recomienda utilizar el gestor de archivos de **Colab** para subir el archivo y acceder a él. El archivo debe contener las URLs en líneas distintas. Para ver un ejemplo puede acceder al repositorio de [Github](https://github.com/javier-fraga-garcia/pagespeed-auditor) y ver la carpeta `examples`.
#@markdown * `domain` &rarr; la dirección dentro de la cual se quieren realizar las auditorías, por ejemplo `https://www.example.com`. El archivo con las URLs puede contener páginas que no interese auditar porque se ha obtenido mediante web scraping o de alguna fuente que ha retornado URLs que apuntan a sitios que no queremos auditar.
#@markdown * `api_key_file` &rarr; la ruta al archivo con la clave de API para conectarse al servicio de Google PageSpeed. El archivo debe ser en formato de texto plano y solo contener la clave de API. Para ver un ejemplo puede acceder al repositorio de [Github](https://github.com/javier-fraga-garcia/pagespeed-auditor) y ver la carpeta `examples`. Para más información sobre como obtener una clave API consultar la documentación de [Google](https://developers.google.com/speed/docs/insights/v5/get-started#APIKey). 

#@markdown ---
urls_file = '/content/urls.txt' #@param {type: 'string'}
domain = 'https://quotes.toscrape.com' #@param {type: 'string'}
api_key_file = '/content/token.txt' #@param {type: 'string'}

### Setup
Ejecutar las celdas siguientes para configurar el entorno

In [ ]:
%%capture
!wget -N https://raw.githubusercontent.com/javier-fraga-garcia/pagespeed-auditor/main/utils.py
import random
from utils import get_urls, get_api_key, make_audits, make_parallel_audits, write_results

In [ ]:
urls = get_urls(urls_file, domain)
api_key = get_api_key(api_key_file)

### PageSpeed

#### Muestreo

Desplegar esta sección en caso de querer aplicar muestreo.

In [ ]:
#@markdown Ejecutar la celda para saber el número de URLs únicas a auditar
print(f'There are {len(urls)} unique URLs to audit')

In [ ]:
#@markdown Si no se desea auditar todas las urls puede indicar se puede realizar un muestreo sobre ellas para obtener un subconjunto. Indique los valores correspondientes en las siguientes variables en caso de querer aplicar esta operación y ejecute la celda.
#@markdown * `n_sample` &rarr; el número de URLs que se desean conservar para la auditoría.

#@markdown ---
n_sample = 5 #@param {type: 'integer'}

if n_sample > 0 and n_sample < len(urls):
  urls = random.sample(urls, n_sample)
  print(f'Generated sample of {n_sample}')
else:
  print('Sampling has not been applied')

#### Strategy

In [ ]:
#@markdown Seleccione el dispositivo en el que desea realizar la auditoría (`mobile`, `desktop`). 

strategy = 'desktop' #@param ['desktop', 'mobile']

#@markdown * Seleccione el nombre del archivo de resultados (no indique extensión del archivo), por ejemplo `results-mobile`. Cuando la auditoría haya terminado se generará un archivo `csv` con este nombre en el sistema de archivos de **Colab**.

res_file_name = 'results_desktop2' #@param {type: 'string'}

#### Auditorías secuenciales

In [ ]:
#@markdown Ejecutar esta celda para comenzar las auditorías. Se irá generando una traza para seguir el proceso. 

#@markdown Al final se reportará también información sobre el tiempo de ejecución y se generará un archivo con los resultados con el nombre indicado en la sección inicial.
%%time
audits = make_audits(urls, api_key, strategy)
write_results(audits, res_file_name)

#### Auditorías multi-hilo

El código siguiente permite una paralelización de las solicitudes a la API. Esto permite mejorar los tiempos de ejecución y reducir el tiempo que se tarda en realizar las auditorías. Es importante tener en cuenta que si se realizan demasiadas peticiones puede que la API retorne un error o que se supere el límite de cuota.

In [ ]:
%%time
#@markdown Ejecutar la siguiente celda para ejecutar las peticiones paralelizadas. 

#@markdown *NOTA: solo se recomienda en casos con muchas URLs a auditar y cuya ejecución se demore mucho*

#@markdown Indique el número de hilos que desea
threads = 8 #@param{type: 'slider', min:2, max:10, step:1}

audits = make_parallel_audits(urls, api_key, strategy, threads)
write_results(audits, res_file_name)